In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#사용한 ipynb들 정리하기
#update_df
#은미4브랜드_5축제
#BDA_한정_선할인_콜라보_설날_새벽_더마켓only_앱전용특가_깜짝특가_박스_유통기한_세트_오특_행사_패밀리데이
#데이터대분류매핑
#중분류_노가다
#대분류 count df 생성

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup

import re

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전
#데이터셋 불러오기 
df = pd.read_csv('tmk_bda_train.csv', index_col = 0, encoding = 'utf-8')

#브랜드 사전 불러오기 
brand = pd.read_csv('brand_complete.csv')

%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/새벽하스...
#대분류 불러오기
big = pd.read_csv('bigcategory.csv', index_col=0)

#중분류 불러오기 
mid = pd.read_csv('middlecategory.csv', index_col=0)

/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전
/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전/새벽하스...


# 기본 변수

In [ ]:
######두번째 변수: 총 주문금액
  # 주문량 x 주문금액 후 > scd 별로 모두 더함 
df['total_order_amt'] = df['net_order_qty'] * df['net_order_amt']

total_amt_data = pd.DataFrame(df.groupby('scd').sum())['total_order_amt']

df.drop('total_order_amt', axis=1, inplace=True)
df = pd.merge(df, total_amt_data, on='scd')


######네번째 변수: 브랜드
#데이터프레임을 맵핑할 dictionary로 바꾸기 
bdic = dict(zip(brand['product_name'], brand['brand']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
df['brand'] = df['product_name'].map(bdic).fillna(np.nan)



######다섯 번째 변수: festival
df['festival'] = 0
df['order_date']= df['order_date'].astype(int)

# def func(x):
#  if x < 11:
#    return 1
#  else:
#    return 0

# #10일 기준으로 이진분류

for i in range(len(df)):
 if (str(df['order_date'][i])[-2:] in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']):
  df['festival'][i] = 1

######여섯번째 변수 : salary
df['salary'] = 0

for i in range(len(df)):
 if (df['employee_yn'][i] == 'N') and (str(df['order_date'][i])[-2:] in ['10', '11', '12', '25', '26', '27']):
   df['salary'][i] = 1

for i in range(len(df)):
  if (df['employee_yn'][i] == 'Y') and (str(df['order_date'][i])[-2:] in ['21', '22', '23']):
    df['salary'][i] = 1



In [ ]:
# ######첫번재 변수: 고객 정보 
# # 10대, 2030F/M, 4050F/M, 60대의 6개 분류

for i in range(len(df)):
  if df.iloc[i,5] == 1:
    df.loc[i,'age_gen'] = '10 FM'
  elif (df.iloc[i,5] == 2) | (df.iloc[i,5] == 3):
    if df.iloc[i,4] == 'F':
      df.loc[i,'age_gen'] = '2030 F'
    else:
      df.loc[i,'age_gen'] = '2030 M'
  elif (df.iloc[i,5]== 4) | (df.iloc[i,5]==5):
    if df.iloc[i,4] == 'F':
      df.loc[i,'age_gen'] = '4050 F'
    else:
      df.loc[i,'age_gen'] = '4050 M'
  else:
    df.loc[i,'age_gen'] = '60 FM'

# [] 변수 생성

In [ ]:
#product_name은 수정용. real_product은 원래용꺼. 
df['real_product'] = df['product_name']

In [ ]:
df['product_name'].value_counts()

비비고 사골곰탕 500g                                                      305
비비고 사골곰탕 500gX18개(1box)                                            254
[500개한정]고메 중화짬뽕 652gX2개                                            244
고메 중화짬뽕 652g                                                       240
비비고 수제진한김치만두 400gx2                                                235
                                                                  ... 
[27%선할인적용] 이너비 글루타치온 필름 400mgX30매X2개                                 1
[2023설사원선물신청]정품진 한우 상차림 세트 1.8kg                                     1
고메치킨 후라이드550gX2개                                                     1
[새벽배송][쿡킷] 킬바사 투움바 파스타                                               1
비비고 특양지곰탕 700g+비비고 왕교자 1.05kgX2개+비비고 남도떡갈비 450g+실리만 볶음주걱+실리만 집게      1
Name: product_name, Length: 3113, dtype: int64

In [ ]:
######한정, 선할인, 콜라보, 설날, 새벽배송 등등등 

#[이벤트] 들고오기 
event_list = []
product_list = df['product_name'].unique()

for product in product_list:

  start_idx = product.find('[')
  end_idx = product.find(']')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      event_list.append(extracted_value)

event_list = list(set(event_list))

In [ ]:
###변수 생성
df['limit'] = 0 #한정
df['pre_dcnt'] = 0 #선할인
df['colab'] = 0 #콜라보
df['new_years'] = 0 #설날 세트, 특가, 캠페인 등 설날 관련 상품
df['early_del'] = 0 # 새벽배송
df['exclusive'] = 0 # CJ더마켓ONLY 상품
df['dcnt'] = 0 # 앱전용특가, 깜짝특가
df['box'] = 0 # 박스
df['mid_cat2'] = 0 #중분류 

In [ ]:
# 00개 한정 

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '한정' in value:
        df['limit'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 선할인

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '선할인' in value:
        df['pre_dcnt'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 콜라보

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '침착맨' in value:
        df['colab'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
#설날 세트, 특가, 캠페인 등 설날 관련 상품

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '23설' in value:
        df['new_years'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

      if 'CJ공식몰_23설선물세트특가' in value:
        df['new_years'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 새벽배송

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '새벽배송' in value:
        df['early_del'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# CJ더마켓ONLY 상품

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if 'ONLY' in value:
        df['exclusive'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 앱전용특가

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '앱전용특가' in value:
        df['dcnt'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 깜짝특가

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '깜짝특가' in value:
        df['dcnt'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 박스

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if 'BOX' in value:
        df['box'][i] = 1
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
# 대괄호값 이용하여 중분류 채우기

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '택배배송' in value:
        df['mid_cat2'][i] = '1001'
      if '쿡킷' in value:
        df['mid_cat2'][i] = '1002'

In [ ]:
# 제거 

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if value in event_list:
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
event_list = []
product_list = df['product_name'].unique()

for product in product_list:

  start_idx = product.find('[')
  end_idx = product.find(']')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      event_list.append(extracted_value)

event_list = list(set(event_list))
event_list # 빈 리스트인지 확인!!!!!!!!!!!

['[택배배송]', '[냉동]', '[사은품]', '[식물성]', '[쿡킷]', '[bibigo table]']

In [ ]:
# 한번 더 돌려주기 -> 대괄호값 이용하여 중분류 채우기

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '택배배송' in value:
        df['mid_cat2'][i] = '1001'
      if '쿡킷' in value:
        df['mid_cat2'][i] = '1002'

In [ ]:
# 제거 

for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if value in event_list:
        new_product_name = product_name.replace(value, '')
        df['product_name'][i] = new_product_name

In [ ]:
event_list = []
product_list = df['product_name'].unique()

for product in product_list:

  start_idx = product.find('[')
  end_idx = product.find(']')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      event_list.append(extracted_value)

event_list = list(set(event_list))
event_list # 빈 리스트인지 확인!!!!!!!!!!!

[]

## () 값 파생변수

In [ ]:
small_list = []
product_list = df['product_name'].unique()

for product in product_list:

  start_idx = product.find('(')
  end_idx = product.find(')')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      small_list.append(extracted_value)

small_list = list(set(small_list))
small_list

['(2개월)',
 '(12입)',
 '(6개)',
 '(용기)',
 '(유통기한:2023-05-22)',
 '(유통기한:2023-06-30)',
 '(6입x2묶음)',
 '(유통기한23.01.29까지)',
 '(쯔란갈비1ea/마장반면1ea/납육초반1ea)',
 '(새콤한맛)',
 '(80gx10입)',
 '(미향)',
 '(중)',
 '(유통기한 23.04.24)',
 '(초이스 등급)',
 '(메론2,샤인머스캣1 3.5kg)',
 '(3입)',
 '(100g*10)',
 '(10미)',
 '(특상)',
 '(1-2인분X5개입)',
 '(4마리분)',
 '(전복내장영양밥 200gX2개+불고기버섯영양밥 200gX2개+뿌리채소영양밥 200gX2개+꿀약밥 200gX2개)',
 '(27G*40개)',
 '(유통기한23.01.28까지)',
 '(T)',
 '(45gx6봉)',
 '(1box)',
 '(유통기한23.05.23까지)',
 '(300g*2팩)',
 '(감바스,나시고랭,불고기,치킨김치,멕시코식,바비큐포크)',
 '(60g*10입)',
 '(유통기한:2023-08-26)',
 '(220gX5입)',
 '(2인)',
 '(망고파인)',
 '(유통기한:2023-10-01)',
 '(간장숙성)',
 '(100g*7입)',
 '(유통기한:2023-09-09)',
 '(쯔란갈비1ea/마장반면1ea/납육초반1ea/실리만집게1ea)',
 '(유통기한:23.07.02)',
 '(꼬지)',
 '(유통기한23.02.28까지)',
 '(1등급 이상 1.8kg)',
 '(유통기한 23.03.22)',
 '(총600g)',
 '(4인분)',
 '(50g*20입)',
 '(2인분)',
 '(유통기한:2023-05-29)',
 '(2BOX)',
 '(와사비맛)',
 '(유통기한23.05.28까지)',
 '(구이용)',
 '(꼬치)',
 '(유통기한:2023-04-23)',
 '(냉동)',
 '(4주)',
 '(80g*10입)',
 '(유통기한23.02.17까지)',
 '(유통기한 23.

In [ ]:
df['expir'] = 0 # 유통기한 임박 제품

In [ ]:
# 유통기한 임박 제품(한번 더 해줘야 함)
for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('(')
  end_idx = product_name.find(')')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '유통기한' in value:
        df['expir'][i] = 1
       # new_product_name = product_name.replace(value, '')
       # df['product_name'][i] = new_product_name

In [ ]:
for i, product_name in enumerate(df['product_name']):

  start_idx = product_name.find('(')
  end_idx = product_name.find(')')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if ('box' in value) | ('BOX' in value) | ('박스' in value):
        df['box'][i] = 1
       # new_product_name = product_name.replace(value, '')
       # df['product_name'][i] = new_product_name

## 세트 상품

In [ ]:
set_list = []

for i in range(len(df)):
  if '+' in df['product_name'][i]:
    set_list.append(df['product_name'][i].strip())

set_list = list(set(set_list))

In [ ]:
substring_list = ['리턴업', '루테인']

# Use list comprehension to filter out values that contain the substring
for substring in substring_list:
  filtered_list = [value for value in set_list if substring not in value]

filtered_list

['주부초밥왕 새콤달콤320g+160gx2개',
 '정품진 무항생제 1+등급 한우 등심 200gx3팩_패밀리데이',
 '정품진 무항생제 1+등급 한우 등심 200g+채끝 200g_패밀리데이',
 '삼호 매콤달콤 쫀득어묵볶이 330g+삼호 쫀득어묵 500g+삼호 가쓰오모둠어묵탕413gX2개',
 '양고기 선물세트 2호 (숄더랙600gx2팩+시즈닝2종)',
 '미트마스터 한돈 삼겹살300g+목심살300g+갈매기살300g',
 '양고기 선물세트 1호 (숄더랙600g+프렌치랙600g+시즈닝2종)',
 '고메 페퍼로니토마토 시카고피자 390g+고메 클래식 콤비네이션 피자 405g+고메 클래식 불고기 피자 405g',
 '햇반솥반 미미달 기획세트(전복내장영양밥 200gX2개+불고기버섯영양밥 200gX2개+뿌리채소영양밥 200gX2개+꿀약밥 200gX2개)',
 '더건강한 닭가슴살 직화 통살구이100gX3개+직화 스테이크100gX3개+소시지120gX3개',
 '비비고 묵은지900g+비비고 석박지 900g',
 '가쓰오우동2인+얼큰우동2인x2개',
 '리턴업 뼈건강케어 칼슘+마그네슘+비타민D(1개월)X2개',
 '고메 트러플 크림파스타 740g+고메 머쉬룸 투움바 파스타 740g+고메 봉골레 오일파스타 626g',
 '맛밤3+맛군밤3 혼합번들',
 '비비고 고등어구이 60g+소스15gX2개',
 '미트마스터 한돈 삼겹살300g+목심살300g+항정살300g+등심덧살300g_패밀리데이',
 '행복한콩 반모두부(150g+150g)',
 '비비고 한마리고등어구이 140gx2개 + 비비고 갈치구이 70gx2개 + 비비고 고등어구이 60gx2개',
 '밸런스밀 귀리 50gX2개+밸런스밀 견과 50gX2개+밸런스밀 고구마 50gX2개+밸런스밀 바나나 50gX2개',
 '백설 리얼 매실청 310mlx2개+ 초정탄산수 500ml',
 '백설 스모크 치킨&포크 바베큐소스 315g+그릴드 씨푸드&베지 바베큐소스 285g+치폴레 스파이시 디핑소스 320g(유통기한:23-04-23)',
 '햇반쿡

In [ ]:
# 세트 상품 파생변수

df['set'] = 0
for i in range(len(df)):
  if df['product_name'][i] in filtered_list:
    df['set'][i] = 1

In [ ]:
df[df['set'] == 1]

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,pre_dcnt,colab,new_years,early_del,exclusive,dcnt,box,mid_cat2,expir,set
18,20230124155563,행복한콩 반모두부(150g+150g),1,7.333676,M,3,Y,20230124,N,147.475732,...,0,0,0,0,0,0,0,0,0,1
57,20230125157569,비비고 고등어구이 60g+소스15g,1,8.225771,M,4,Y,20230125,Y,155.858295,...,0,0,0,0,0,0,0,0,0,1
93,20230125160689,가쓰오우동2인+얼큰우동2인x2개,1,9.482807,M,3,Y,20230125,Y,142.913545,...,0,0,0,0,0,0,0,0,0,1
128,20230125156838,햇반솥반 미미달 기획세트(전복내장영양밥 200gX2개+불고기버섯영양밥 200gX2개...,2,10.115813,F,5,N,20230125,N,122.438808,...,0,0,0,0,0,0,0,0,0,1
129,20230125156838,햇반소프트밀 전복죽 420gx3개+단호박죽 420gx3개,1,9.184202,F,5,N,20230125,N,122.438808,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45486,20230128188435,더건강한 닭가슴살 직화 통살구이100gx2개+갓츄 핫소스 샘플키트,11,10.907679,F,2,Y,20230128,Y,119.984472,...,0,0,0,0,0,0,0,0,0,1
45559,20230103990248,더건강한 닭가슴살 직화 스테이크100gX5개+갓츄 핫소스 샘플키트,1,9.420520,F,4,N,20230103,N,49.950178,...,0,0,0,0,0,0,0,0,0,1
45848,20230101966556,더건강한 닭가슴살 직화 통살구이100gX3개+직화 스테이크100gX3개+소시지120...,1,10.033726,F,3,Y,20230101,N,10.033726,...,0,0,0,0,0,0,0,0,0,1
45866,20230101963968,비비고 특양지곰탕 700g+비비고 왕교자 1.05kgX2개+비비고 남도떡갈비 450...,1,10.832300,M,5,N,20230101,Y,10.832300,...,0,0,0,0,0,0,0,0,0,1


## _오특, _행사, _패밀리데이 파생변수

In [ ]:
df['event'] = 0

for i, product_name in enumerate(df['product_name']):
  if '_오특' in product_name:
    df['dcnt'][i] = 1
  if ('_행사' in product_name) or ('패밀리데이' in product_name):
    df['event'][i] = 1

In [ ]:
## []까지 제거한거: product_name
## ()와 정규식 모두 제거한거: prod_processed

df['prod_processed'] = df['product_name']

## 정규식 제거

In [ ]:
df.head()

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,new_years,early_del,exclusive,dcnt,box,mid_cat2,expir,set,event,prod_processed
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,Y,20230124,N,92.801653,...,0,0,0,0,0,0,0,0,0,잔칫집 식혜 240ml 30입
1,20230124153976,고메 칠리감바스 피자 350g,1,9.103089,F,2,Y,20230124,N,92.801653,...,0,0,0,0,0,0,0,0,0,고메 칠리감바스 피자 350g
2,20230124153976,고메 포테이토치즈 핫도그400g,1,9.151969,F,2,Y,20230124,N,92.801653,...,0,0,0,0,0,0,0,0,0,고메 포테이토치즈 핫도그400g
3,20230124153976,고메 소보로 고르곤졸라 시카고 피자 370g,1,9.103089,F,2,Y,20230124,N,92.801653,...,0,0,0,0,0,0,0,0,0,고메 소보로 고르곤졸라 시카고 피자 370g
4,20230124153976,고메 클래식 불고기 피자 405g,1,8.879472,F,2,Y,20230124,N,92.801653,...,0,0,0,0,0,0,0,0,0,고메 클래식 불고기 피자 405g


In [ ]:
prod_name = df.loc[:,'product_name']

def remove_square_brackets(text):
    result = re.sub(r'\[.*?\]', '', text)
    return result.strip()


# 리스트의 모든 요소에 대해 괄호와 그 안의 문자를 삭제
prod_name = [remove_square_brackets(item) for item in prod_name]
len(prod_name)


def extract_patterns(lst):
    patterns = []
    for text in lst:
        pattern = re.search(r'[A-Za-z가-힣\s]+', text)
        if pattern:
            patterns.append(pattern.group().strip())
        else:
            patterns.append('')
    return patterns


refined_prod_name = extract_patterns(prod_name)

df['prod_processed'] = refined_prod_name

# middle_cat(중분류) 전처리

product_name: [] 제거된거
 
prod_processed: 모두 다 제거된거 

In [ ]:
mid.head()

,product_name,middle,prod_processed
0,해찬들 사계절쌈장 500g,4010,해찬들 사계절쌈장
1,해찬들 새콤달콤 초고추장 1050g,4010,해찬들 새콤달콤 초고추장
2,해찬들 고기찍어먹는장 찍장 쌈장 300g,4010,해찬들 고기찍어먹는장 찍장 쌈장
3,해찬들 궁중식 쇠고기 볶음고추장 500g,4010,해찬들 궁중식 쇠고기 볶음고추장
4,해찬들 100% 우리쌀 태양초 고추장 1kg,4010,해찬들


In [ ]:
#브랜드 사전 
middledic = dict(zip(mid['prod_processed'], mid['middle']))

#브랜드명 부여
df['middle_cat'] = df['prod_processed'].map(middledic).fillna(np.nan)

### 1. 전처리 안된 값이랑 비교하여 중분류 넣기
mid_id_list = ['0004010', '0004011', '0004012', '0004013', '0004014', '0004015', '000407', '000408', '000409', \
               '000501', '000502', '000503', '000505', '000601', '000602', '000603', '000604', '000702', '000703', '000705',\
               '000901', '000902', '000903', '000904', '000905', '001001', '001002', '001003', \
               '001801', '001802', '001803', '001804', '001805', '001806', '001807', '001901', '001903', '001904', '001905', '001906', '001907', \
               '002001', '002002', '002003', '002004', '002005', '002101', '002102', '002104', '002105', '002201', '002202', '002203', \
               '002301', '002302', '002303', '002304']

mid_data = {} 
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/중분류

for id in mid_id_list:
  with open(id + '.txt', 'r', encoding='cp949') as file:
    lines = [line.strip() for line in file]
    unique_lines = list(set(lines)) # 중복값 제거
    mid_data[id] = unique_lines

df['mid_cat2'] = df['product_name'].apply(lambda x: next((id for id in mid_id_list if any(item in x for item in mid_data[id])), 0))


### 2. 전처리한 중분류값이랑 비교하여 값 채우기

df['middle_cat'] = df['middle_cat'].fillna(0).astype(int) # 대분류 값 정수형으로 변환
df['mid_cat2'] = df['mid_cat2'].astype(int)



#브랜드 사전
middledic = dict(zip(mid['prod_processed'], mid['middle']))

#대분류명 부여
for i in range(len(df)):
  if df['mid_cat2'][i] == 0:
    df['mid_cat2'][i] = df['middle_cat'][i]

df[df['mid_cat2'] == 0]['prod_processed'].value_counts()


### 3. 은미가 전처리한거 반영
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/이거다
eunmi = pd.read_csv('thisisdata.csv')
eunmi = eunmi.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

for i in range(len(df)):
  if df['mid_cat2'][i] == 0:
    df['mid_cat2'][i] = eunmi['middle_cat2'][i]


### 4. 노가다
search_dic = {'빅반들':1802, '크레잇 콰트로 콘맥앤치즈':2102, '숯불향고추장삼겹살':904, '삼호 가쓰오 모둠 어묵탕':1906, '비비고 순살고등어구이':1904, \
              '비비고 언양식바싹불고기':2203, '미초':501, '얼큰우동':1805, 'VIPS':1003, '유산균':601, '이너비':604, '쁘띠첼':2104, '특별한선택':2301, '유러피안':414, '통삼겹스테이크':2302, '리턴업':603, '드레싱':407, '행복한콩':1906, \
                   '사골순댓국':1901, '참기름':414, '스팸고급유':2301, '핫도그':2101, '죽':1804, '소스':407, '국수':1805, '닭가슴살 통살':2304, \
                    '컵반': 1802, 'CJ명가':1907, '빕스':1003, '흑삼':602, '최고의선택':2301, '스팸스위트':2301, '쿡킷':1002, 'BIG BEST':1802, '믹스':409, \
                    '화다닭':2304, '스팸듬뿍부대찌개':1002, '스팸':2301, 'Black Label':2301, '올리브유':414, 'EXCYCLE':2105, '식혜':502, '스모크 치킨':407, \
                     'BYO':601, '맛밤':2104, '맥스봉':2101, '빅반들':1802, '크레잇 콰트로 콘맥앤치즈':2102, '숯불향고추장삼겹살':904, '삼호 가쓰오 모둠 어묵탕':1906, \
              '순살고등어구이':1904, '언양식바싹불고기':2203, '비엔나':2303, '스파게티':1805, '아이시스':502, '백설 프리미엄':414, '칠성사이다':502, \
              '그레인보울':1807,'다담':408, '닭가슴살':2304, '햄스빌 베이컨':2302, '베이컨':2302, '된장찌개양념':410, '된장찌개 양념':410, '만두':2001, \
                '그릴후랑크':2303, '투썸':505, '한우':703, '쌈장':410, '숯불향 치즈닭갈비':1002, '한뿌리':602, '쿠키':2102, '생선조림양념':408, \
                '직화구이 백설너비아니':2203, '씨푸드':407, '백설':414, '삼호':1906, '찹스테이크':1002, '두부':1906, '밸런스밀':1807, 'BIG TRAY':1802, \
                '숯불향':1002, '도시락':1807, '트레비':502, '계절밥상':1003, '감바스':1002, '채끝 구이용':703, '이츠웰': 702, '프레시웨이 LA갈비':703, '프레시웨이':702, \
                '정육':703, '쌈무':1905, '하선정':412, '호주산':703, '오리훈제':703, '포도씨유':414, '동물복지':1906, \
                '샐러드':702,'두부면':1906,'면':1805, '국':1901, '솜사탕':2102, '그라탕':2001, '해신탕':1001, '김치':1903, '유정란':1905, \
                '치킨':2003,'너겟':2003, '닭한마리':1001, '초반': 2004, '커피':505, '돈까스':2201, '양고기 선물세트':703, '쯔란':1002, '고기':1002, \
                '김말이':2102, '고등어':1904, '아이누리':502, '삼천포':1002, '파스타':1805, '고추장':410, 'BLACKLABEL':2301, '천혜향':702, \
                '샤인머스캣':702, '레드향':702, '자몽':702, '양배추':602, '배':702, '골드키위':702, '체리':702, '한정SET':2004, '간짜장':2004, '시럽':503, \
                '서울우유': 2105, '불낙전골':1002, '버섯':702, '정품진':703, '살치살':703, '찜갈비':703, '견과':1807, '제주':705, '굴비':705, \
                '한과':2102, '들깨':1002, '부채살스테이크':1002, '살':703, '보일링랍스터':1001, '프리미엄쌀':702, '치즈파우더':407, '달고나':2102, \
                '멸치':705, '겉절이':1002, '평양식 어복쟁반':1002, '새우까스':2201, '튀김':2201, '흑화고세트':702, '건강쌀':702, '몽골리안비프':1002, \
                  '풀드포크':2302, '아보카도':702,'깐쇼새우':2004,'망고선물세트':702}

keys = search_dic.keys()

for key in keys:
  for i in range(len(df)):
    if df['mid_cat2'][i] == 0:
      if key in df['prod_processed'][i]:
        # print(df['product_name'][i] + '////' + df['prod_processed'][i])
        df['mid_cat2'][i] = search_dic[key]


### 마지막 처리

for i in range(len(df)):
  if df['mid_cat2'][i] == 4010:
    df['mid_cat2'][i] = 410
  elif df['mid_cat2'][i] == 4011:
    df['mid_cat2'][i] = 411
  elif df['mid_cat2'][i] == 4012:
    df['mid_cat2'][i] = 412
  elif df['mid_cat2'][i] == 4013:
    df['mid_cat2'][i] = 413
  elif df['mid_cat2'][i] == 4014:
    df['mid_cat2'][i] = 414
  elif df['mid_cat2'][i] == 4015:
    df['mid_cat2'][i] = 415
  else:
    continue

/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전/중분류
/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전/이거다




---



# main_cat (대분류)

In [ ]:
df['main_cat2'] = df['mid_cat2'] // 100
df['main_cat2'].unique()

array([ 5, 20, 21, 19, 23, 22,  4, 18,  6, 10,  7,  9,  0])

In [ ]:
df.drop(['middle_cat'], axis = 1, inplace = True)

In [ ]:
df.head(3)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,early_del,exclusive,dcnt,box,mid_cat2,expir,set,event,prod_processed,main_cat2
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,Y,20230124,N,92.801653,...,0,0,0,0,502,0,0,0,잔칫집 식혜,5
1,20230124153976,고메 칠리감바스 피자 350g,1,9.103089,F,2,Y,20230124,N,92.801653,...,0,0,0,0,2002,0,0,0,고메 칠리감바스 피자,20
2,20230124153976,고메 포테이토치즈 핫도그400g,1,9.151969,F,2,Y,20230124,N,92.801653,...,0,0,0,0,2101,0,0,0,고메 포테이토치즈 핫도그,21


# weight, capacity 

In [ ]:
# 정규식 패턴
pattern = r'(\d+(?:\.\d+)?(?:kg|mL|mg|ml|g|MG|G|ML|L|KG|Kg))'
# 정규식을 이용하여 매칭되는 데이터 추출
df['matches'] = df['product_name'].str.extractall(pattern).groupby(level=0).first()[0]
# 문자열 분할 및 추출
weight = df.loc[:,['product_name','matches']]



def process_matches(value):
    match = re.match(r'(\d+(\.\d+)?)([a-zA-Z]+)', value)
    if match:
        num = match.group(1)
        unit = match.group(3)
        if unit in ('mg', 'MG'):
            return 0
        elif unit in ('g','G', 'mL','ml','ML'):
            return num
        elif unit in ('KG', 'Kg', 'kg', 'L'):
            return float(num) * 1000
    return 0


weight['weight'] = weight['matches'].apply(str).apply(process_matches)
weight['weight'] = weight['weight'].astype(float)



weight['capacity'] = 0 


weight.loc[weight['weight'] >= 1000, 'capacity'] = 1  # 조건을 만족하는 경우 1 할당


kilogram = weight[['product_name','weight','capacity']]

In [ ]:
kilogram.isna().sum()

product_name    0
weight          0
capacity        0
dtype: int64

In [ ]:
kdic = dict(zip(kilogram['product_name'], kilogram['capacity']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
df['capacity'] = df['product_name'].map(kdic).fillna(np.nan)

In [ ]:
wdic = dict(zip(kilogram['product_name'], kilogram['weight']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
df['weight'] = df['product_name'].map(wdic).fillna(np.nan)

In [ ]:
df.head(3)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,box,mid_cat2,expir,set,event,prod_processed,main_cat2,matches,capacity,weight
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,Y,20230124,N,92.801653,...,0,502,0,0,0,잔칫집 식혜,5,240ml,0,240.0
1,20230124153976,고메 칠리감바스 피자 350g,1,9.103089,F,2,Y,20230124,N,92.801653,...,0,2002,0,0,0,고메 칠리감바스 피자,20,350g,0,350.0
2,20230124153976,고메 포테이토치즈 핫도그400g,1,9.151969,F,2,Y,20230124,N,92.801653,...,0,2101,0,0,0,고메 포테이토치즈 핫도그,21,400g,0,400.0


# 일반회원과 임직원 분리하기

In [ ]:
emp_df = df[df['employee_yn'] == 'Y'].reset_index(drop=True)
reg_df = df[df['employee_yn'] == 'N'].reset_index(drop=True)

# 임직원 선물하기 변수

In [ ]:
#임직원인 경우에만 해당됨 코드가 넘 오래걸려서 맨 마지막 변수로 추가하자 ㅇ
#일곱번째 변수: 선물하기 유무 

 #선물하기 카테고리 크롤링
url = 'https://www.cjthemarket.com/pc/gift/giftProd?wid1=lnb_gift'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
html = requests.get(url, headers = headers)
soup = BeautifulSoup(html.text, "html.parser")

gift_items = soup.find_all('p', class_='product-title')

gift_item_list = []


for item in gift_items:
  item_name = item.text
  item_name = item_name.strip()
  gift_item_list.append(item_name)


emp_df['gift'] = 0
for i in range(len(emp_df)):
  if emp_df['product_name'][i] in gift_item_list:
    for j in range(len(emp_df)): # 해당 주문번호에 해당하는 row 모두 gift == 1
      if emp_df['scd'][j] == emp_df['scd'][i]:
        emp_df['gift'][j] = 1

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/입실렌티전야제
import os
os.getcwd()

/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전/입실렌티전야제


'/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전/입실렌티전야제'

In [ ]:
reg_df.to_csv('reg_train_v4.csv')
emp_df.to_csv('emp_train_v4.csv')

In [ ]:
#reg_df.to_csv('reg_train.csv')

In [ ]:
emp_df.head()

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,mid_cat2,expir,set,event,prod_processed,main_cat2,matches,capacity,weight,gift
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,Y,20230124,N,92.801653,...,502,0,0,0,잔칫집 식혜,5,240ml,0,240.0,0
1,20230124153976,고메 칠리감바스 피자 350g,1,9.103089,F,2,Y,20230124,N,92.801653,...,2002,0,0,0,고메 칠리감바스 피자,20,350g,0,350.0,0
2,20230124153976,고메 포테이토치즈 핫도그400g,1,9.151969,F,2,Y,20230124,N,92.801653,...,2101,0,0,0,고메 포테이토치즈 핫도그,21,400g,0,400.0,0
3,20230124153976,고메 소보로 고르곤졸라 시카고 피자 370g,1,9.103089,F,2,Y,20230124,N,92.801653,...,2002,0,0,0,고메 소보로 고르곤졸라 시카고 피자,20,370g,0,370.0,0
4,20230124153976,고메 클래식 불고기 피자 405g,1,8.879472,F,2,Y,20230124,N,92.801653,...,2002,0,0,0,고메 클래식 불고기 피자,20,405g,0,405.0,0


# 참고 코드

In [ ]:
######test set 돌릴 때 안 돌려도 되는 코드 #############
#####just 참고용

#brand에서 brand_complete로 코드 구현 방법 
brand = pd.read_csv('/content/drive/MyDrive/BDA 공모전/brand_dic.csv')

#"The더건강한"은 "더건강한"으로 바꾸어 저장 (제품명에 The를 안붙이는 경우가 있기에-brand1 열 생성 관련)
brand.loc[brand['brand'] == 'The더건강한', 'brand'] = '더건강한'
brand.loc[brand['brand'] == '삼호어묵', 'brand'] = '삼호'

brand = brand.drop(columns=['Unnamed: 2',	'Unnamed: 3'])


#데이터프레임을 맵핑할 dictionary로 바꾸기
bdic = dict(zip(brand['product'], brand['brand']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
df['brand_dic'] = df['product_name'].map(bdic).fillna(np.nan)


## nan 개수 33760개, 너무 많아서 이를 해결할 방법 필요 
  # 제품명에 '비비고, '햇반', '고메' 등의 브랜드명 포함

# 'product_name'에 20개의 브랜드명 중 하나가 포함되면, 그 브랜드명을 <brand>열에 저장
target_strings = ['비비고', '햇반', '고메', '백설', '더건강한', '스팸', '다시다', '해찬들',
                  '바이오코어', '한뿌리', '삼호', '하선정', '행복한콩', '다담', '크레잇',
                  '메티에', '쁘띠첼', '맥스봉', '맛밤',
                  'BYO', '명가', '쿡킷', '리턴업', '이츠웰', '닥터뉴트리', '즐거운동행', '미트마스터 한돈', '밸런스밀', '아이누리',
                  '플랜테이블', '육공육', 'EXCYCLE']

# 'brand' 열 생성: 위 브랜드명 문자열 포함 여부 확인하여 저장 (없으면 nan)
df['brand'] = df['product_name'].apply(lambda x: next((s for s in target_strings if s in x), np.nan))
df.loc[df['product_name'].str.contains('빕스|아이시스|코카|VIPS|투썸|칠성|쿠캣|얼티브|그린너트'), 'brand'] = '타브랜드'
### 최고의 선택|특별한 선택
#df[df['brand'].isna()]

#brand열 nan 10687개로 줄음. (brand_dic열 nan 33760개)
##brand열이 nan인 case를,  브랜드사전을 이용한 brand_dic열로 채워줄 수 있음
## 따라서 1) 둘이 일치하지 않는 경우 중에서 
## 2)둘 다 nan이 아니고, 3) brand_dic열이 nan이 아닌 경우 찾아서 채워주기


#'brand_dic' 열과 'brand' 열 값이 다른 경우를 확인
mask = (df['brand_dic'] != df['brand'])

  #그 중에 두 열 모두 NaN이 아닌 경우만 필터링
mask_both_not_nan = ~(df['brand_dic'].isna() & df['brand'].isna())

  #조건에 맞는 행 출력
result = df[mask & mask_both_not_nan]

# +brand_dic 열이 nan이 아닌 경우
mask_brand_not_nan = ~df['brand_dic'].isna()

  #조건에 맞는 행 출력
result = df[mask & mask_both_not_nan & mask_brand_not_nan]
# 923개!

# 기존 DF에서 'result' 데이터프레임의 인덱스와 일치하는 행의 'brand' 열을 'brand_dic'열 값으로 채워주기
df.loc[result.index, 'brand'] = df['brand_dic']

nobrand = df[df['brand'].isna()]

#결측치 9764개로 줄음
#10687(기존 brand1 결측치) - 923(brand1은 결측치지만 브랜드사전엔 있는 case) = 9764개

# '[택배배송]제품명' 형태로 이루어진 obs 중 <제품명> 부분이 브랜드 사전에 있을경우 브랜드 넣어주기
import re
for key, value in bdic.items():
    df.loc[df['product_name'].str.contains(re.escape(key)), 'brand'] = value

brackets = nobrand[nobrand['brand'].isna()==0]

df.loc[brackets.index, 'brand'] = brackets['brand']

nobrand = df[df['brand'].isna()]

In [ ]:
newdic = {'잔칫집 식혜 240ml 30입': '롯데', '가쓰오얼큰우동2인분': 'CJ', '사골곰탕 한 그릇 100g (1-2인분X5개입)': '백설', '밀당의고수 쫄면 4인':'CJ',
          '꿀 고구마 2kg (특상)_패밀리데이':'타브랜드', '꿀 고구마 2kg (중)' : '타브랜드',
       '밥이랑야채 27g':'백설', '제주 삼다수 500ml 40입 (20입x2묶음)':'타브랜드', '밀당의고수 바삭한 김말이 400g':'CJ',
       '[임직원]가쓰오우동 4인분933.2gx2개':'CJ', '발사믹드레싱 250g': 'CJ', '토종김 참기름 5 g x 12 EA': '명가',
       '가쓰오우동 4인분 933.2g':'CJ', '가쓰오우동 2인분 466.6g':'CJ', '사계절쌈장 500g':'해찬들',
       '산들애 처음부터 멸치다시마 육수 275g':'산들애', '깐메추리알 450g':'CJ',
       '[500개한정]가쓰오우동 4인분 933.2gX2개':'CJ', '[500개한정]탱글탱글 불쭈꾸미볶음_밀키트X2개':'쿡킷',
       '[임직원]미정당 스파이시로제떡볶이 376gx2개':'즐거운동행', '훈제대란 20구':'CJ', '[택배배송] 탱글탱글 불쭈꾸미볶음_밀키트':'쿡킷',
       '미정당비엔나떡볶이300g(용기형)X2개':'즐거운동행', '[임직원]이너비 콜렉티브 콜라겐 930mgX84정(4주)X6개' : '이너비',
       '중화요리간짜장 2인분 630g':'CJ', '[임직원]동치미물냉면 4인분x2개': 'CJ', '[임직원]얼큰우동2인분x2개':'CJ',
       '[임직원]베트남 쌀국수2인분375gx2개':'CJ', '주부초밥왕 새콤달콤 꼬마 149.2g':'CJ',
       '전립소 아르기닌 6000(15일분)X2개': 'CJ', '[임직원]동치미냉면육수300gx2개':'CJ',
       '[임직원]가쓰오우동2인+튀김우동2인x2개':'CJ', '이너비 콜렉티브 콜라겐 930mgx84정(4주)X2개' : '이너비',
       '이너비 아쿠아뱅크 300mgX56캡슐(4주)X2개' : '이너비', '[500개한정]꼬마새우까스 250gx2개':'CJ',
       '[임직원]주부초밥왕 한입사각유부초밥400gx2개':'CJ', '[500개한정] CJ 꼬마돈까스 1.2kgX2개':'CJ',
       '오이피클머스타드소스245g':'하선정', '[임직원]주부초밥왕 새콤달콤유부초밥 320gx2개':'CJ',
       '[임직원]생야채돼지고기물만두400g*2eax2개':'CJ', '밀당의고수 바삭한 김말이 400gX2개':'CJ',
       '[임직원]씨제이 납작군만두700g':'CJ', '[임직원]CJ 꼬마돈까스1.2kgx2개':'CJ', '두부면 중화짜장면 2인분': '즐거운동행',
       '가쓰오우동 2인분+튀김우동 2인분X2개':'CJ', '[임직원]가쓰오우동2인+얼큰우동2인x2개':'CJ',
       '[임직원]가쓰오우동한그릇x2개':'CJ', '[임직원]가쓰오우동 2인분 466.6gx2개':'CJ', '통단무지 400g':'하선정',
       '정통식 야끼소바 2인분 412g':'CJ', '[임직원]이너비_스노우화이트 500MG*56캡슐(4주)x2개' : '이너비',
       '가쓰오메밀생면1136.8g(4인분)':'CJ', '[임직원]밀당의고수쫄면2인x2개':'CJ', '[임직원]깐메추리알 450gx2개':'CJ',
       '꼬마너겟 320g':'CJ', 'CJ 동물복지 유정란10구':'CJ', '[임직원]CJ 교자만두1.41kg':'CJ',
       '베트남 쌀국수 2인분 375g':'CJ', '[500개한정]꼬마 돈까스 400gx2개':'CJ', '가쓰오모둠어묵탕413gx2개':'CJ',
       '정말맛있는소시지160g':'백설', '[임직원]까르보나라스파게티2인분x2개':'CJ',
       '[500개한정]미정당 매콤까르보나라 누들떡볶이X2개':'즐거운동행', '[임직원]두부면 들기름 간장파스타 2인분x2개':'즐거운동행',
       'CJ 납작군만두 700g':'CJ', '[700개한정]햄스빌 굿베이컨130g*2X2개':'백설', '햄스빌굿베이컨130g*3':'백설',
       '주부초밥왕 한입사각유부초밥400g':'CJ', '[임직원]함흥비빔냉면소스85gx5개':'CJ', '[임직원]감자수제비가루500gx2개':'백설',
       '햄스빌 베이컨 160gX2개':'백설', '해쉬브라운 냉동감자 1.2kg':'타브랜드', '미정당 스파이시로제떡볶이 376g':'즐거운동행',
       '산들애 국내산 한우 250g':'산들애', '오천년의 신비 명품구운소금 400g':'백설', '오리엔탈 드레싱 105g':'CJ',
       '미정당 생칼국수 1kg':'즐거운동행', '토레타 500ml x 24개':'타브랜드', '맛소금 250g':'백설',
       '가쓰오우동 2인분+튀김우동 2인분':'CJ', '[200개한정]밀당의고수 바삭한 김말이 400gX2개':'CJ',
       '주부초밥왕 한입사각유부초밥400gx2개':'CJ', '반숙대란 20구':'CJ', '볼로냐스파게티2인625g':'CJ',
       '베트남 쌀국수 2인분 375gX2개':'CJ', '[100개한정]병아리콩 그레인보울 160gx2개':'햇반',
       '[임직원]사천마라탕면 2인분 434gx2개':'CJ', '[500개한정]숯불향 치즈닭갈비&볶음밥_밀키트X2개':'쿡킷',
       '[500개한정]가쓰오튀김우동 2인분x2개':'CJ', '[500개한정]가쓰오얼큰우동2인분x2개':'CJ',
       '[1000개한정미정당 스파이시로제떡볶이 376gx2개':'즐거운동행', '[임직원]정통식 야끼소바 2인분 412gx2개': 'CJ',
       '주부초밥왕 새콤달콤320g+160g': 'CJ', '[임직원]밀당의고수 김말이400gx2개':'CJ', '우동 면사리 190g':'CJ',
       '미정당 방앗간 가래떡 떡볶이 400gX2개':'즐거운동행', '얼큰우동한그릇 용기X2개':'CJ', '주부초밥왕 새콤달콤꼬마 447.6g': 'CJ',
       '사천마라탕면 2인분 434gX2개':'CJ', '[600개한정] 햄스빌 베이컨160gX2X2개':'백설',
       '[임직원]CJ 꼬마새우까스 250Gx2개':'CJ', '[임직원][냉동] 탱글탱글 불쭈꾸미볶음_밀키트x2개':'쿡킷',
       '[500개한정]주부초밥왕 한입사각유부초밥400gX2개':'CJ', '[500개한정]우동 면사리 190gx2개':'CJ',
       '[임직원]5가지 재료로만 만든 부침가루 1KG':'백설', 'CJ 교자만두1.41kg':'CJ',
       '[500개한정] 정통식 야끼소바 2인분 412gx2개':'CJ', '가벼운 고단백 두부면(넓은면)X2개':'즐거운동행',
       '미정당 방앗간 가래떡 떡볶이 400g':'즐거운동행', '꼬마새우까스 250g':'CJ', '시저드레싱 235g':'CJ',
       '제주 삼다수 2L 12입 (6입x2묶음)':'타브랜드', '펩시 제로슈거 355ml X 24캔_패밀리데이':'타브랜드',
       '아몬드 호두 드레싱 230g':'CJ', '레몬 1.4kg(120gX12입)':'타브랜드', '동치미물냉면 4인분':'CJ',
       '동치미냉면육수 300g':'CJ', '꿀 고구마 2kg (중)_패밀리데이':'타브랜드', '딜라이트 프로젝트 레몬라임 눈꽃 솜사탕': '타브랜드',
       '명란시대 짜지않고 맛있는 명란젓 800g':'타브랜드', '오리엔탈드레싱260g':'CJ', '사과스위트칠리소스270g':'CJ',
       '까르보나라스파게티_2인분_600g':'CJ', '가쓰오튀김우동 2인분':'CJ', '딜라이트 프로젝트 눈꽃 달고나': '타브랜드',
       '[500개한정]숯불향 고추장삼겹살 구이_밀키트X2개':'쿡킷', '[택배배송] 쯔란 돼지고기 볶음_밀키트':'쿡킷',
       '오곡참깨 드레싱105g':'CJ', '햄스빌 굿베이컨130g*2':'백설', '핫케익믹스1kg':'백설', '임직원 생일 선물':'CJ',
       '미정당 매콤까르보나라 누들떡볶이X2개':'즐거운동행', '베이컨로제스파게티2인분_641g':'CJ', '꼬마 돈까스 400g':'CJ',
       '[임직원]햄스빌 굿베이컨130g*2x2개':'백설', '[임직원][냉동] 숯불향 치즈닭갈비&볶음밥_밀키트x2개':'쿡킷',
       '[임직원]미정당 매콤까르보나라 누들떡볶이x2개':'즐거운동행', '주부유부왕 300g': 'CJ', '[임직원]오곡참깨드레싱 240gx2개':'CJ',
       'CJ 꼬마너겟 750g':'CJ', '[임직원]오리엔탈드레싱260gx2개':'CJ', '[임직원]찹쌀호떡믹스400Gx2개':'백설',
       '[임직원]팻다운 파워번 스페셜 가르시니아 100mlx10병 x2개':'CJ', '가쓰오우동한그릇X2개':'CJ',
       '[2023설사원선물신청] 특별한선택 1호' :'백설', '가쓰오우동2인+얼큰우동 2인':'CJ',
       '트레비 탄산수 레몬 500ml 18입':'타브랜드', '울트라레귤러컷 냉동감자 2kg':'타브랜드', '영양균형 핫케익믹스 400g':'백설',
       '정품진 무항생제 1+등급 한우 등심 200g+채끝 200g_패밀리데이':'정품진',
       '정품진 무항생제 1+등급 한우 등심 200gx3팩':'정품진', '정품진 1+ 등심 300g+갈비살 300g':'정품진',
       '미정당 매콤까르보나라 누들떡볶이':'즐거운동행', '동치미물냉면 2인분':'CJ', '사천마라탕면 2인분 434g':'CJ',
       '오천년의 신비 명품천일염(중간입자) 500g':'백설', '두부면 토마토파스타 2인분':'즐거운동행', 'CJ 동물복지 유정란15구':'CJ',
       '[27%선할인적용] 이너비 글루타치온 필름 400mgX30매X2개' :'이너비', '전립소 아르기닌 6000 (15일분)':'CJ',
       '주부초밥왕 국산콩 328 g': 'CJ', '[택배배송] 스페인식 감바스_밀키트':'쿡킷',
       '[임직원]베이컨로제스파게티2인분641gx2개':'CJ', '[500개한정]주부유부왕 300gx2개': 'CJ',
       '두부면 들기름 간장파스타 2인분':'즐거운동행', '주부초밥왕 새콤달콤유부초밥 320g': 'CJ', '오곡참깨드레싱 240g':'CJ',
       'CJ 꼬마돈까스 1.2kg': 'CJ', '[임직원]중화요리 간짜장2인분630gx2개':'CJ', '가쓰오우동 4인분 933.2gX2개':'CJ',
       '이너비 콜렉티브 콜라겐 930mgx84정(4주)':'이너비', '양갈비 프렌치랙 300g+시즈닝 2종':'타브랜드', '깐메추리알 270g':'CJ',
       '[임직원]전립소 아르기닌 6000 (15일분)X12개':'CJ',
       '[임직원]주부초밥왕 새콤달콤 유부초밥 도시락기획280gx2개': 'CJ', '비요뜨 아이스크림 3종(딸기, 블루베리, 초코볼)':'타브랜드',
       '[임직원]칠리토마토 스파게티 640g 2인분x2개':'CJ',
       '[26%선할인적용] 이너비 글루타치온 필름 400mgX30매X2개':'이너비', '밥이랑 해물 24g':'백설', '밥이랑치즈 27g':'백설',
       '플리또 골드키위배 130ml':'이츠웰', '고소한 볶음참깨 430g':'이츠웰', '주부초밥왕 국산콩 328g': 'CJ',
       '미정당 어묵떡볶이346g':'즐거운동행', '무항생제 닭다리 400g 2팩':'CJ', '[택배배송] 숯불향 치즈닭갈비&볶음밥_밀키트':'쿡킷',
       '칠레산 생 체리 800g내외':'타브랜드', '유자샐러드소스 250g':'CJ', '스타루비 자몽 3.3kg (10과 내외)':'타브랜드',
       '밀당의고수 쫄면 2인X2개':'CJ', '[100개한정]할라피뇨 그레인보울 160gx2개':'햇반',
       '산들애 처음부터 멸치다시마 육수 525g':'산들애', '베트남 쌀국수 2인분 375gx2개':'CJ',
       '이너비 글루타치온 필름 400mgX30매':'이너비', '[임직원]발사믹드레싱 250gx2개':'CJ',
       '영양균형 핫케익믹스 400gX2개':'백설', '[임직원]이너비아쿠아뱅크300mgx56캡슐(4주)x3개':'이너비',
       '[임직원]주부초밥왕 새콤달콤320g+160gx2개': 'CJ', '미정당 어묵떡볶이346gX2개':'즐거운동행',
       '[임직원]함흥비빔냉면2인분x2개':'CJ', '[택배배송]트러플 부채살 찹스테이크_밀키트':'쿡킷',
       '[500개한정]CJ 교자만두1.41kgX2개':'CJ', '주부초밥왕 새콤달콤320g+160gX2개': 'CJ',
       '[500개한정][택배배송] 블랙페퍼 찹스테이크_밀키트X2개':'쿡킷', '정통식 야끼소바 2인분 412gX2개':'CJ',
       '함흥비빔냉면소스 85g':'CJ', '[임직원][택배배송] 육수가득 소불고기 전골_밀키트':'쿡킷',
       '이너비 글루타치온 필름 400mgX30매X2개':'이너비', '[임직원]블랙페퍼 찹스테이크(2인)_밀키트':'쿡킷',
       '양갈비 어깨살 300g_패밀리데이':'타브랜드', '양갈비 프렌치랙 300g_패밀리데이':'타브랜드',
       '[CJ더마켓ONLY] 한정SET#1 (쯔란갈비1ea/마장반면1ea/납육초반1ea/실리만볶음주걱1ea)':'고메',
       '4無 치자 이절 단무지 400g':'하선정', '가벼운 고단백 두부면(얇은면) 100gx2개':'즐거운동행',
       '[임직원]햄스빌굿베이컨130g*3x2개':'백설', '가쓰오우동 4인분 933.2gx2개':'CJ',
       '[CJ더마켓ONLY] 한정SET#2 (쯔란갈비1ea/마장반면1ea/납육초반1ea/실리만집게1ea)':'고메',
       '[눈건강] 아이시안 루테인 아스타잔틴60+(1개월)':'CJ', '파인애플 샐러드소스 250g': 'CJ', '1+등급 건강란 15구':'CJ', '[피부건강] 이너비 콜렉티브 콜라겐 930mgx84정(1개월)':'이너비', '칠리토마토 스파게티 640g 2인분': 'CJ', '트레비 탄산수 라임 500ml 18입':'타브랜드', '감자수제비가루500g':'백설', '무항생제 아랫날개(윙) 350g 2팩':'타브랜드',
'녹차맛 찹쌀호떡믹스 400 g':'백설', '에비앙 330ml x 24입_패밀리데이':'타브랜드', '정품진 무항생제 1+등급 한우 양지 200g':'정품진', '컨디션환 20포':'CJ', '무항생제 윗날개(봉) 350g 2팩':'타브랜드',       '[임직원]가쓰오튀김우동2인분x2개':'CJ', '트레비 ECO 무라벨 탄산수 350ml x 20개 (플레인)':'타브랜드', '계절밥상 숙성담은 고추장 불고기 500gx2팩':'계절밥상', '펩시 제로슈거 355ml X 24캔':'타브랜드', 
'꿀 고구마 2kg (특상)':'타브랜드', '살치살 구이용 200g 2팩(미국산,초이스등급)_패밀리데이':'타브랜드','물엿1.2kg':'백설', '왕새우튀김 300g(10미)':'타브랜드', '아티장 밀가루 티(T)55 2kg':'타브랜드', '함흥비빔냉면 2인분':'CJ','가쓰오우동 2인분 466.6gX2개':'CJ', '[임직원]미정당 어묵떡볶이346g(용기형)x2개':'즐거운동행', '부채살 구이용 200g 2팩 (미국산, 초이스등급)_패밀리데이':'타브랜드', '비요뜨 아이스크림 3종(딸기, 블루베리, 초코볼)_패밀리데이':'타브랜드', '동물복지 유정란15구':'CJ',
       '컨디션 헛개 100mlx2병':'CJ', '가쓰오우동한그릇':'CJ', '옥사부반점 사천식류산슬250gX3개_패밀리데이':'CJ',
       '키위 샐러드소스 250g':'CJ', '바삭군만두 445g*2EA':'비비고', '[Q팩] 더마켓 BEST':'CJ',
       '계절밥상 마늘간장쪽갈비 500gx2팩':'계절밥상', '베이컨로제스파게티2인분 641gX2개':'CJ', '밀당의고수 쫄면 4인X2개':'CJ',
       '오이피클머스타드소스595g':'CJ', '동그랑땡200g*2':'백설', '[2023설사원선물신청]송이고버섯선물세트 700g':'타브랜드',
       '제주 삼다수 330ml x 40입':'타브랜드', '[2023설선물세트]청도 반건시 선물세트 1.5kg':'타브랜드',
       '[2023설사원선물신청]영광 민어굴비 세트 1.7kg':'타브랜드', '[2023설사원선물신청] 특별한선택 THE호':'백설',
       '[2023설선물세트] 특별한선택 T호':'백설', '[2023설선물세트] 특별한선택 O-1호(스프레이오일세트)':'백설',   
           '[2023설사원선물신청] 특별한선택 스페셜 O호':'백설', '[2023설사원선물신청] 특별한선택 2호X4':'백설',
       '[2023설사원선물신청] 특별한선택 O호':'백설', '컨디션헛개수 340mlx20개(1박스)':'CJ', '요거트 드레싱245g':'CJ',
       '무항생제  닭가슴살 400g 2팩':'타브랜드', '[2023설사원선물신청] 특별한선택 NE호':'CJ',
       '[23설 임직원 단독]특별한선택 1호X4개':'백설', '[2023설사원선물신청] 특별한선택 S호':'백설',
       '[CJ공식몰_23설선물세트특가]특별한선택 THE호x3개':'백설', '[2023설사원선물신청] 특별한선택 K-2호X4':'백설',
       '[깜짝특가][CJ공식몰_23설선물세트특가]특별한선택 K-2호x4개':'백설', '[2023설사원선물신청] 특별한선택 K-2호':'백설',
       '[2023설사원선물신청] 구증구포 흑삼대보 골드40mlx30포':'CJ',
       '[깜짝특가][CJ공식몰_23설선물세트특가]특별한선택 NE호x4개':'백설',
       '[2023설사원선물신청] 구증구포 흑삼진 블랙스틱 10gx30포':'CJ',
       '[23설 임직원 단독]구증구포 흑삼진 블랙스틱 10gx30포X6개':'CJ', '[2023설선물세트]포도씨유 3호':'타브랜드',
       '[2023설사원선물신청] 구증구포 흑삼진 스틱 10gx30포':'CJ', '통감자 바삭부침가루 1Kg':'백설',
       '[2023설사원선물신청] 흑삼정 골드클래스 100gx2':'타브랜드', '[CJ공식몰_23설선물세트특가]특별한선택 O-G호x4개':'백설',
       '[2023설사원선물신청]오리훈제 선물세트 1.6kg':'타브랜드', '제일제면소 생칼국수550g':'백설',
       '[2023설사원선물신청]프레시웨이 사과,배,샤인머스캣 선물세트 5kg':'타브랜드',
       '[2023설사원선물신청]소찜갈비 정육 선물세트 3.5kg(초이스 등급)':'타브랜드',
       '[2023설사원선물신청]프레시웨이 배 세트 5.4kg':'타브랜드',
       '[2023설선물세트]소찜갈비 정육 선물세트 3.5kg(초이스 등급)':'타브랜드', '[2023설사원선물신청]레드향 세트 3kg':'타브랜드',
       '[2023설선물세트]프레시웨이 한라봉, 레드향, 천혜향 세트 3.6kg':'타브랜드',
       '헬씨누리 침향환 환심 10환X6입(1BOX)_패밀리데이':'헬씨누리', '헬씨누리 침향환 환심 10환':'헬씨누리', '밀당의고수 쫄면 2인':'CJ',
       '[2023설선물세트]프레시웨이 사과, 배 세트 7.5kg':'타브랜드', '[2023설사원선물신청]천혜향 세트 3kg':'타브랜드',
       '주부초밥왕 새콤달콤 유부초밥 도시락기획280g':'CJ', '[택배배송] 숯불향 고추장삼겹살 구이_밀키트':'쿡킷',
       '[2023설사원선물신청] 특별한선택 스페셜 O호X4':'백설', '오이피클머스타드소스245gX2개':'CJ',
       '[2023설선물세트] 흑삼 젠스틱 10gx30포':'타브랜드', '[2023설사원선물신청] 특별한선택 2호':'CJ',
       '바질&로즈마리드레싱 235gX2개':'CJ', '[2023설사원선물신청]천혜향, 레드향, 애플망고 세트 3.5kg':'타브랜드',
       '[깜짝특가][CJ공식몰_23설선물세트특가]특별한선택 O-1호(스프레이오일세트) x3개':'백설',
       '요거트 드레싱245gX2개':'CJ', '이너비 글로우샷 콜라겐(6개)X2개':'이너비',
       '[CJ공식몰_23설선물세트특가]최고의선택 특호x3개':'백설', '발사믹드레싱 250gX2개':'CJ',
       '[23설 임직원 단독]구증구포 흑삼진 스틱 10gx30포X6개':'CJ', '오곡참깨 드레싱105gX2개':'CJ',
       '[2023설선물세트]유러피안 S호':'백설', '[2023설사원선물신청] 특별한선택 O-1호(스프레이오일세트)':'백설',
       '키위 샐러드소스 115gX2개':'CJ', '[2023설선물세트]특별한선택 1호':'백설',
       '팻다운 파워번 마스터 가르시니아 100mlX10개':'CJ',
       '[2023설사원선물신청]정품진 한우 GOURMET 상차림용 세트 2.4kg':'정품진',
       '[2023설사원선물신청]정품진 1등급 한우 불고기,국거리세트 1.8kg':'정품진',
       '[2023설선물세트]정품진 1등급 한우 불고기,국거리세트 1.8kg':'정품진',
       '[2023설사원선물신청]제주은갈치 선물세트 1.2kg':'타브랜드', '[2023설사원선물신청]프리미엄 흑화고세트 500g':'타브랜드',
       '건강을 생각한 요리유 900ml':'백설', '얼큰우동한그릇 용기':'CJ', '서리태드레싱240gX2개':'CJ',
       '아메리칸 핫도그용 포크소시지 950g':'CJ', '정품진 무항생제 친환경 1+등급 한우 양지 200g_패밀리데이':'정품진',
       '태양의 식후비법 W차 500ml x 24개':'타브랜드', '서울우유 흰우유 아이스크림 474ml 3개_패밀리데이':'타브랜드',
       '녹차맛 찹쌀호떡믹스 400g':'CJ', '[2023설사원선물신청]샤인머스캣 선물세트 2kg내외 (3입)':'타브랜드',
       '[2023설사원선물신청] 특별한선택 O-1호(스프레이오일세트) X3':'백설', '청양레몬간장 드레싱 245g':'CJ',
       '[냉동] 블랙페퍼찹스테이크_밀키트X2개':'쿡킷', '[2023설사원선물신청] 최고의선택 특호':'백설',
       '[2023설선물세트]특별한선택 K-2호':'백설',
       '[침착맨 SET] BIG TRAY - 바질크림2ea/트러플크림2ea/매콤로제2ea/차돌된찌1ea/돼지고기김찌1ea':'CJ',
       '[앱전용특가] 아이시안 루테인 지아잔틴40 X2개 (유통기한:23.07.02)':'CJ', '오리엔탈드레싱260gX2개':'CJ',
       '[2023설선물세트]유러피안 M호':'백설', '[23설 임직원 단독]흑삼정 골드클래스 100g*2X3개':'CJ',
       '[2023설사원선물신청] 특별한선택 12호 X5':'백설', '[2023설사원선물신청] 하이프로틴 유산균 10일분X3':'BYO',
       '[CJ공식몰_23설선물세트특가]특별한선택 스페셜 O호x4개':'백설', '[2023설임직원캠페인]특별한선택 2호':'백설',
       '[2023설사원선물신청] 특별한선택 12호':'백설', '[2023설사원선물신청]청도 반건시 선물세트 1.5kg':'타브랜드',
       '[2023설사원선물신청]장수세대 멸치3종세트 960g':'타브랜드', '잔칫집 식혜 240ml 30입_오특':'타브랜드',
       '샤인머스캣 포도 2kg내외 (3~4입)':'타브랜드', '[2023설선물세트]청도 반건시 프리미엄 선물세트 1.5kg':'타브랜드',
       '정품진 무항생제 1+등급 한우 등심200gx2팩+채끝200g':'정품진',
       '[2023설사원선물신청]정품진 한우 GOURMET 상차림용 세트 1.8kg':'정품진',
       '정품진 무항생제 1+등급 한우 양지 200g+설도 200g':'정품진',
       '[2023설사원선물신청]제스프리 골드키위 선물세트 2.1kg':'타브랜드',
       '[2023설사원선물신청]찜갈비 선물세트 1.6kg(호주산)':'타브랜드',
       '[2023설사원선물신청]청도 반건시 프리미엄 선물세트 1.5kg':'타브랜드',
       '[2023설사원선물신청]정품진 한우알찬세트(1등급 이상 1.8kg)':'정품진',
       '[2023설사원선물신청]담양한과 한과선물세트(미향)':'타브랜드',
       '[2023설선물세트]프레시웨이 LA갈비 선물세트 3.2kg_오특':'타브랜드',
       '[2023설사원선물신청]정품진 한우 구이 세트1.8kg':'정품진', '[앱전용특가] 스페인식 감바스_밀키트':'쿡킷',
       '이너비 스노우화이트 500mgX56캡슐(4주)X2개':'이너비', '[2023설선물세트]건강쌀 4종선물세트 1kg':'타브랜드',
       '[앱전용특가]이너비 아쿠아뱅크 300mgX56캡슐(4주)X2개':'이너비', '[2023설선물세트]특별한선택 2호':'백설',
       '[2023설사원선물신청] 특별한선택 스페셜 The호':'백설', '[2023설사원선물신청] Black Label':'타브랜드',
       '[2023설사원선물신청] 구증구포 흑삼진 골드스틱10gx30':'CJ', '무농약 대추방울토마토 1kg':'타브랜드',
       '[임직원]쉐프솔루션 치킨가라아게1kgx2개':'CJ', '[2023설임직원캠페인] 특별한선택 O-1호(스프레이오일세트)':'백설',
       '[2023설사원선물신청] 양배추&브로콜리 골드 80mlx30입X2':'타브랜드',
       '[2023설사원선물신청] 특별한선택 NE호X4':'백설', '오천년의 신비 명품천일염(가는입자) 250g':'백설',
       '고소한 볶음참깨 1kg':'백설', '채끝 구이용 200g 2팩(미국산,초이스등급)_패밀리데이':'타브랜드',
       '[냉동] 탱글탱글 불쭈꾸미볶음_밀키트X2개':'쿡킷', '[2023설사원선물신청]프레시웨이 LA갈비 선물세트 3.2kg':'타브랜드',
       '[2023설사원선물신청]호주산 LA 갈비 1.8kg':'타브랜드', '[2023설선물세트] 구증구포 흑삼진 스틱 10gx30포':'CJ',
       '[2023설선물세트]특별한선택 O호':'백설', '[2023설사원선물신청]구워만든 곡물그대로 21 블랙에디션':'타브랜드',
       '컨디션 헛개수 340ml':'CJ', '초코칩 쿠키믹스 290g':'백설',
       '[2023설사원선물신청] 양배추&브로콜리 골드 80mlx30입':'타브랜드',
       '[2023설사원선물신청] 흑삼 젠스틱 10gx30포':'타브랜드', 'CJ 발사믹드레싱 610g':'CJ',
       '[냉동] 숯불향 고추장삼겹살 구이_밀키트X2개':'쿡킷', '[임직원]팻다운 부스터 카르니틴 6입x10개':'CJ',
       '[임직원][냉동] 숯불향 고추장삼겹살 구이_밀키트x2개':'쿡킷', '[임직원]동치미물냉면2인분x2개':'CJ',
       '이너비 리바이탈 세라마이드 600mgX56정(4주)X2개':'이너비', '강력 밀가루 20kg':'백설',
       '토레타 500ml x 24개_패밀리데이':'타브랜드', '[2023설선물세트] 특별한선택 THE호':'백설',
       '[2023설사원선물신청] 구증구포 흑삼대보 골드40ml 30포X6':'CJ',
       '[2023설선물세트]양고기 선물세트 2호 (숄더랙600gx2팩+시즈닝2종)':'타브랜드',
       '[2023설선물세트]양고기 선물세트 1호 (숄더랙600g+프렌치랙600g+시즈닝2종)':'타브랜드',
       '[2023설임직원캠페인특가]최고의선택 특호x3개':'백설', '[23설 임직원 단독]특별한선택 THE호X3개':'백설', '펩시 제로슈거 355ml X 24캔_오특': '타브랜드', '[2023설선물세트] 특별한선택 NE호': '백설',
       '[2023설선물세트] 구증구포 흑삼진 블랙스틱 10gx30포': 'CJ', '[2023설임직원캠페인] 특별한선택 THE호': '백설',
       '[2023설임직원캠페인] 흑삼정 골드클래스 100gx2': 'CJ', '[2023설선물세트]최고의선택 특호': '백설',
       '[2023설사원선물신청] 특별한선택 T호': '백설', '[2023설임직원캠페인] 구증구포 흑삼진 블랙스틱 10gx30포': 'CJ',
       '[2023설사원선물신청]프레시웨이 사과 세트 3.8kg': '타브랜드',
       '[2023설선물세트]이스라엘 메르쥴 품종 대추야자 선물세트 700g': '타브랜드',
       '[2023설사원선물신청]프레시웨이 한라봉, 레드향, 천혜향 세트 3.6kg': '타브랜드',
       '서울우유 흰우유 아이스크림 474ml 3개': '타브랜드', '[2023설선물세트]찜갈비 선물세트 1.6kg(호주산)': '타브랜드',
       '[2023설사원선물신청]이스라엘 메르쥴 품종 대추야자 선물세트 700g': '타브랜드',
       '[2023설임직원캠페인]프레시웨이 사과 세트 3.8kg': '타브랜드', '컨디션헛개수 500mlx20개(1박스)': '타브랜드',
       '바삭한 빵가루 1kg': '백설', '태양의 식후비법 W차 500ml x 24개_패밀리데이': '타브랜드',
       '생야채 돼지고기 물만두 400x2개': '타브랜드', '[2023설사원선물신청]혼합 버섯세트': '타브랜드',
       '계절밥상 마늘간장쪽갈비 500gx2팩_오특': '계절밥상', '[2023설선물세트]천혜향 세트 3kg': '타브랜드',
       '비요뜨 아이스크림 블루베리x3개': '타브랜드', '[2023설사원선물신청] 최고의선택 프리미엄호': '백설', '꼬마새우까스 250gX2개': 'CJ',
       '[2023설선물세트]레드향 세트 3kg': '타브랜드', '계절밥상 숙성담은 간장 불고기 500gx2팩': '계절밥상',
       '옥사부반점 사천식류산슬250gx3개': 'CJ', '식빵믹스760g': '백설',
       '[CJ공식몰_23설선물세트특가]구증구포 흑삼진 스틱 10G*30포x2개': 'CJ', '청양레몬간장 드레싱 245gX2개': 'CJ',
       '[2023설선물세트] 특별한선택 스페셜 O호': '백설', '탱글탱글 불쭈꾸미볶음_밀키트': '쿡킷',
       '[2023설사원선물신청]제주산 옥돔&갈치 혼합세트 830g': '타브랜드', '살치살 구이용 200g 2팩(미국산,초이스등급)': '타브랜드',
       '이너비 아쿠아리치 더블업 600mgX56캡슐(4주)': '이너비',
       '[2023설사원선물신청]한우,한돈 구이용 혼합세트 1.8kg': '타브랜드', '까르보나라스파게티 2인분 600gX2개': 'CJ' ,
       '[2023설임직원캠페인]특별한선택 O호': '백설', '[2023설선물세트]장수세대 멸치3종세트 960g': '타브랜드',
       '[2023설사원선물신청]정품진 한우 상차림 세트 1.8kg': '정품진',
       '[2023설사원선물신청]영광법성포 참조기 굴비세트 900g': '타브랜드',
       '[2023설선물세트]프레시웨이 LA갈비 선물세트 3.2kg': '타브랜드', '정품진 1+ 갈비살 600g(300gX2팩)': '정품진',
       '컨디션 헛개수 1.5L': '타브랜드', '정품진 무항생제 1+등급 한우 안심 200g+등심 200g+채끝 200g(구이용)': '정품진',
       '미정당비엔나떡볶이300g(용기형)': '즐거운동행',
       '[2023설사원선물신청]양고기 선물세트 1호 (숄더랙600g+프렌치랙600g+시즈닝2종)': '타브랜드',
       '[2023설선물세트]담양한과 한과선물세트(죽향)': '타브랜드', '오리엔탈 드레싱 105gX2개': 'CJ',
       '팻다운 부스터 카르니틴 6입': 'CJ', '[2023설선물세트]혼합 버섯세트': '타브랜드', '[2023설임직원캠페인]특별한선택 1호': '백설',
       '[2023설임직원캠페인] 구증구포 흑삼진 스틱 10gx30포': 'CJ', 'CJ 오리엔탈드레싱\xa0620g': 'CJ',
       '무항생제 닭볶음용 800g': '타브랜드', '[2023설사원선물신청]메론홍망고선물세트(메론2,홍망고2 3.5kg)': '타브랜드',
       '[2023설임직원캠페인] 구증구포 흑삼대보 골드40mlx30포': 'CJ', '레몬&라임드레싱610g': 'CJ',
       '[임직원]CJ 꼬마너겟750gx2개': 'CJ', '정품진 무항생제 한우차돌박이 200g 2팩': '정품진',
       '[2023설사원선물신청]정품진 한우 1++ 앞다리 스테이크, 국거리 세트 800g': '정품진',
       '[택배배송] 육수가득 소불고기 전골_밀키트': '쿡킷', '[2023설선물세트]구워만든 곡물그대로 21 블랙에디션': '타브랜드',
       '[2023설선물세트]배 선물세트 6.5kg': '타브랜드', '[2023설사원선물신청]담양한과 한과선물세트(죽향)': '타브랜드',
       '새싹보리 500mlX24개_오특': '타브랜드', '바질&로즈마리드레싱 235g': 'CJ',
       '[2023설임직원캠페인] 특별한선택 스페셜 O호': '백설', '[2023설선물세트]영광법성포 참조기 굴비세트 900g': '타브랜드',
       '[2023설선물세트]특별한선택 12호': '백설', '[2023설선물세트대량구매]특별한선택 K-2호': '백설',
       '쯔란 돼지고기 볶음_밀키트': '타브랜드', '[2023설선물세트]메론샤인머스캣선물세트(메론2,샤인머스캣1 3.5kg)': '타브랜드',
       '[2023설사원선물신청]정품진 한우 사골모듬뼈 보신세트 4kg': '정품진', '아메리칸 와플믹스 10kg': '백설',
       '꿀 고구마 2kg (특상)_행사': '타브랜드', '햄스빌 바로먹는햄 등심 140g': '백설', '무항생제 백숙용 생닭 830g': '타브랜드',
       '정품진 무항생제 1+등급 한우 등심 200gx3팩_패밀리데이': '정품진', '산들애 처음부터 가쓰오 육수 525g': 'CJ',
       '꿀 고구마 (중) 2KG_행사': '타브랜드', '중력 밀가루 10kg': '백설',
       '[2023설사원선물신청]정품진 한우 꼬리반골 보신세트 4kg': '정품진', '아티장 밀가루 강력소프트 2kg': '백설',
       '[2023설선물세트]정품진 한우 꼬리반골 보신세트 8kg (4kgx2개)': '정품진', '양갈비 어깨살 300g+시즈닝 2종': '타브랜드',
       '가쓰오튀김우동 2인분X2개': 'CJ', '이너비 아쿠아뱅크 300mgX56캡슐(4주)X6개': '이너비',
       '[앱전용특가] 꼬마돈까스 1.2kg': 'CJ', '그대로 끓여먹는 바지락꽃게 된장찌개 양념 450gX2개': '타브랜드',
       '케이준프라이 냉동감자 2kg': '타브랜드', '[임직원]이너비 콜렉티브 콜라겐 930mgx84정(4주)x2개': '이너비',
       '[임직원]두부면 중화짜장면 2인분x2개': '즐거운동행', '[임직원]미트 볼로냐 스파게티2인 625gx2개': 'CJ' ,
       '컨디션 스틱 그린애플맛 10입': '타브랜드', '[2023설사원선물신청] 흑삼지천보 진녹 30입': 'CJ', '가쓰오튀김우동 2인분x2개': 'CJ',
       '브라우니믹스 320g': '백설', '[앱전용특가] 육수가득 소불고기 전골_밀키트': '쿡킷',
       '[CJ공식몰_23설선물세트특가]특별한선택 스페셜 The호x4개': '백설', '[2023설선물세트]호주산 LA 갈비 1.8kg': '타브랜드',
       '[택배배송] 블랙페퍼 찹스테이크_밀키트': '타브랜드', '랍스터홍새우가리비관자플래터': '타브랜드', '햄스빌굿베이컨130g*3x2개': '백설',
       '제일제면소 생칼국수 300g': '백설', '옥수수식빵믹스 760g': '백설', '[남성건강] 전립소 아르기닌 6000(15일분)': 'CJ',
       '정품진 무항생제 1+등급 한우 등심 200gx2팩': '정품진', '무농약 양파 3kg': '타브랜드',
       '정품진 무항생제  1+ 한우 채끝200g x 2팩_행사': '정품진', '[44%선할인적용]이너비 글로우샷 콜라겐(6개)X3개': '이너비',
       '아보카도 1.1kg(220gX5입)': '타브랜드', '[2023설임직원캠페인]최고의선택 프리미엄호': '백설',
       '계절밥상 마늘간장쪽갈비 500gx2팩_행사': '타브랜드', '옥사부반점 사천식류산슬250gx3개_행사': 'CJ',
       '메르쥴품종 대추야자 350g x2개': '타브랜드', '[2023설임직원캠페인특가]특별한선택 O호x4개': '백설',
       '[앱전용특가] 블랙페퍼찹스테이크(2인)_밀키트': '타브랜드', '모닝두부140gX6개 + 오곡참깨드레싱 105g': 'CJ',
       '[2023설선물세트]BLACKLABEL': '타브랜드', '모닝두부140gx6개 + 오리엔탈드레싱 105g': 'CJ',
       '정품진 1+ 갈비살 600g (300gx2팩)_행사': '정품진',
       '정품진 무항생제 1+등급 한우 등심200gx2팩+채끝200g_행사':'정품진',
       '[임직원][냉동] 쯔란돼지고기볶음 591gx2개': '타브랜드', '부채살 구이용 200g 2팩(미국산,초이스등급)': '타브랜드',
       '레몬&라임드레싱260g': 'CJ', '[CJ공식몰_23설선물세트특가]구증구포 흑삼진 블랙스틱 10G*30x2개': 'CJ',
       '오곡참깨드레싱 240gX2개': 'CJ', '두부면 들기름 간장파스타 2인분X2개': '즐거운동행', '[2023설임직원캠페인]유러피안 P2호': '타브랜드',
       '[2023설사원선물신청] 특별한선택 O-g호': '백설', '이너비 아쿠아리치 더블업 600mgX56캡슐(4주)X2개': '이너비',
       '태양의 식후비법 W차 500ml x 24개_행사': '타브랜드', '칠레산 생 체리 800g내외_행사': '타브랜드',
       '정품진 무항생제 한우차돌박이 200g 2팩_행사': '정품진', '트레비 탄산수 라임 500ml 20입': '타브랜드',
       '[임직원]밥이랑야채27gx20개':'백설', '[임직원]가쓰오메밀생면1136.8g(4인분)x2개': 'CJ',
       'BBQ풀드포크600g(300g*2팩)(유통기한23.04.07까지)': '타브랜드', '비요뜨 아이스크림 딸기x3개_행사': '타브랜드' ,
       '[냉동] 육수가득 소불고기 전골_밀키트X2개': '쿡킷', '[2023설임직원캠페인대량구매] 특별한선택 Y호': '백설',
       '[CJ공식몰_23설선물세트특가]특별한선택 Y호x4개': '백설', '계절밥상 숙성담은 간장 불고기 500gx2팩_행사': '계절밥상',
       '옥사부반점 광동식난자완스250gx3개_행사': 'CJ', '토레타 500ml x 24개_행사': '타브랜드',
       '[2023설사원선물신청] 특별한선택 Y호': '백설', '[23설 임직원 단독]최고의선택 특호X3개': '백설',
       '[2023설선물세트] 특별한선택 S호': '백설',
       '[2023설사원선물신청]정품진 한우 꼬리반골 보신세트 8kg (4kgx2개)': '정품진',
       '비요뜨 아이스크림 3종(딸기, 블루베리, 초코볼)_행사': '타브랜드', '아티장 밀가루 박력마일드 2kg': '백설',
       '제주 은갈치 230g 16토막 (4마리분)': '타브랜드',
       '정품진 무항생제 1+등급 한우 안심 200g+등심 200g+채끝 200g(구이용)_행사': '정품진',
       '바삭한 요리유 500ml': '백설', '서울우유 4종 아이스크림(흰우유,초코,딸기,바나나)_행사': '타브랜드',
       '모닝두부140gx6개+오리엔탈드레싱 105g+두부면 들기름 간장파스타 2인분+두부면 토마토파스타 2인분': 'CJ',
       '헬씨누리 침향환 환심 10환X6입(1BOX)_행사':'CJ',
       '서울우유 4종 아이스크림(흰우유,초코,딸기,바나나)_패밀리데이': '타브랜드', '미정당 스파이시로제떡볶이 376gX2개': '즐거운동행',
       '[2023설사원선물신청]유기농 프리미엄쌀 5종선물세트 2.25kg': '타브랜드', '[2023설선물세트]유러피안 P2호': '백설',
       '팻다운 부스터 카르니틴 50mlX14병X2개': 'CJ', '꼬마 돈까스 400gX4개+꼬마너겟 320gX4개': 'CJ',
       '유자샐러드소스 250gX2개': 'CJ', '[58%선할인적용] 이너비 아쿠아뱅크 300mgX56캡슐(4주)X6개': '이너비'
}

In [ ]:
nobrand['brand'] = nobrand['product_name'].map(newdic).fillna(np.nan)

df.loc[nobrand.index, 'brand'] = nobrand['brand']